# Path Planning part
Once we know where the obstacles are on a given map, we need to be able to find the shortest path between the start and the desired goal. To do this, there exist many different algorithms to find the shortest path. We will explore some of the main ones and choose one accordingly.

## Algorithm choice
1. __Dijkstra algorithm__:
> This algorithm is the simplest to use: it requires to explore iteratively the 8 cells around the current cell and define the shortest distance from the robot to each cell of a grid map. This allows to easily select a path by choosing always choosing the cell with the  This algorithm is however quite restrictive as it allows at best the robot to move in 8 discrete directions. In our case, with the simple map returned from the vision part, this algorithm is quite good but not optimal.

1. __A* algorithm__
> It is very similar to the Dijkstra algorithm, however, after calculating iteratively the distance from the start accounting for the obstacles, one needs to calculate the distance from the goal without considering the obstacles. These two weights will be added together in each cell. The final path will be done by always choosing the lowest weight in the neighboring cells.
_talk complexity_
   
1. __Vision Graph with Dijkstra algorithm__
> This algorithm is no longer based on a grid map to navigate on the map: the obstacles are described in a line based map. This allows for more complex displacement angles and allows for less computation to be done in case the robot needs to diverge from the predefined path. Even if the conversion is more complicated at the start, once the new map is computed, the update of the path is quite simple. This allows the online part of the computation to be short. Therefore recalculating the path on the go is an effective possibility.

So we need to decompose the problem in solvable parts. The first part is to read the map and find the contours of the obstacles. Then, each obstacle needs to be 

## Libraries
The first step in realizing the vision graph is to actually build the map, T

In [ ]:
#!pip install pyvisgraph
#!pip install opencv-python

### General libraries
These are used in many different functions and cells of this jupyter notebook

In [ ]:
import math
import cv2
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import colors
import csv

 link https://github.com/TaipanRex/pyvisgraph

## Public functions

In [59]:
#OFFLINE
def PathPlanning_offline(graph_name, csvPath, savePath, goalPath, goalsavePath, desired_size, 
                         smoothing_factor, radius, epsilon, plot = False, verbose = False):
    '''
    
    The offline part of the path planning part, computes a vision graph from 
    a csv file describing the obstacles and a goal position. The vision graph 
    is stored in graph_name directory 
    
    Variables : graph_name: string, name given to the path where the vision 
                            graph is stored
                csvPath :  string, path to csv file for the obstacles from 
                            the current directory
                savePath : string, path and name of place to save image of
                            the obstacles 
                goalPath : string, path to csv file for the goal from the 
                            current directory
                goalsavePath : string, path and name of place to save image
                                of the goal
                desired_size: array defining the desired size to resize the 
                                img
                smoothing_factor: float, how much the contour will be 
                                    smoothed
                radius: float, radius of the Thymio in mm
                epsilon : float, margin on how much to grow the corners 
                                from the max size of map to avoid getting out
                                of map

                (optionnal)
                savePath: string, path where the image with obstacles is 
                            stored, needs to be specified if plot = True
                radius: int, radius of thymio, needs to be specified 
                        if plot = true
                plot: bool, plot or not the image (default: False)
                verbose: bool, print info about the progress (default: False)
    

    Return : stop, array [2x1], goal position on map
    '''
    
    if plot:
        pt, ax = plt.subplots(2,2, figsize = (20,20), subplot_kw={'aspect': 'equal'})
        plt.tight_layout()
    if verbose: 
        print("Starting...")
        print("\t Converting image from csv to desired size png...")
        
    if plot: 
        plt.subplot(2,2,1, aspect = 'equal')
        plt.xlim([0, desired_size[0]])
        plt.ylim([0,desired_size[1]])
    gray = csv2gray(csvPath, savePath, desired_size, plot)
    
    if verbose: 
        print("\t Image converted. Output saved to: {}".format(savePath))
        print("\t Finding corners of obstacles...")
        
    if plot: 
        plt.subplot(2,2,2, aspect = 'equal')
        plt.xlim([0, desired_size[0]])
        plt.ylim([0,desired_size[1]])
    corners = findCorners(gray, smoothing_factor, plot)
    
    if verbose: 
        print("\t Corners Found.")
        print("\t Finding Goal in the map...")
    
    stop = FindGoal(goalPath, goalsavePath, desired_size) 
    
    if verbose: 
        print("\t Goal found, saved to: {}".format(goalsavePath))
        print("\t Growing obstacles...")
        
    if plot: 
        plt.subplot(2,2,3, aspect = 'equal')
        #plt.xlim([0, desired_size[0]])
        #plt.ylim([0,desired_size[1]])
        
    c = GrowObstacles(corners, radius, desired_size, epsilon, plot)
    if verbose: 
        print("\t Obstacles grown.")
        print("\t Building vision graph...")
        
    BuildGraph(c, graph_name)
    
    if verbose: 
        print("\t Graph built, saved to: {}".format(graph_name))
        print("Done!")
    if plot: 
        plt.subplot(2,2,4, aspect = 'equal')
        plt.xlim([0, desired_size[0]])
        plt.ylim([0,desired_size[1]])
        
        plt.title("All together")
        #plot image
        plt.imshow(gray, aspect = 'equal',origin = 'lower', cmap='gray')
        
        #plot contours
        for k in range(len(corners)):
            x,y = unzipContour(corners[k])
            # add the first element to close the polygon
            x.append(x[0])
            y.append(y[0])
            plt.plot(x, y, color='green', marker='o', linewidth=2, markersize=8)
        
        #plot grown obstacles
        for k in range(len(c)): #-2
            x = c[k][0]
            y = c[k][1]
            x.append(x[0])
            y.append(y[0])
            plt.plot(x,y , color='red', marker='o', linewidth=3, markersize=1)
        
        plt.show()
    return stop


# ONLINE
def PathPlanning_online(graph_name, start, stop, savePath = '', radius = 60, plot = False, verbose = False):
    '''
    
    The online part of the path planning part, recomputes the shortest 
    path from a new start position for a known vision graph graph_name
    
    Variables : graph_name: string, name given to the path where the 
                            vision graph is stored
                start: array [2x1], array giving the coordinates of 
                        the initial position
                stop: array [2x1], array giving the coordinates of 
                        the goal position 
                        
                (optionnal)
                savePath: string, path where the image with obstacles 
                        is stored, needs to be specified if plot = True
                radius: int, radius of thymio, needs to be specified
                        if plot = true
                plot: bool, plot or not the image (default: False)
                verbose: bool, print info about the progress (default: False)

    Return : Path, array [2xM], in order the next coordinates to visit
    '''
    if verbose:
        print("Update starting...")
    path,state = FindShortestPath(graph_name, start, stop)
    if state == 0:
        inObstacle = False
        if verbose:
            print("Update done, path changed")
    else:
        inObstacle = True
        print("Thymio in an obstacle, no new path computed")
        plot = False
        path = []
        
    if plot:
        img = cv2.imread(savePath)
        #fig = plt.figure(figsize=(10,10))
        plt.title("Path to follow")
        plt.imshow(img, aspect = 'equal',origin = 'lower')
        x, y = zip(*path)
        plt.plot(x,y,color = 'blue', linewidth=3)
        plt.plot(start[0], start[1], color = 'green', markersize=radius/2 ,marker='o')
        plt.plot(stop[0], stop[1], color = 'red', markersize=12 ,marker='x')
    
    return path, inObstacle

## Private functions

## Offline functions
All these will be computed before the robot starts to move

### Read CSV file and return grayscale image

In [65]:
import matplotlib.image as mpli

def csv2gray(csvPath, savePath, desired_size, plot = False):
    '''
    Convert a csv file to a grayscale image at desired scale, saved at path savePath
    
    Variables : csvPath : string, path to csv file from the current directory
                savePath : string, path and name of place to save image
                desired_size: array defining the desired size to resize the img
                plot: bool, plot or not the image (default: False)
                
    Return : gray : Grayscale image of the csv file           
    '''
    # from csv file at csvPath, save an image at savePath
    img = np.genfromtxt(csvPath, delimiter=',')
    mpli.imsave(savePath, img, cmap='gray_r')
    img = cv2.imread(savePath)
    
    img = np.flipud(img)
    
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # blur and threshold to merge close polygons
    gray = cv2.blur(gray,(5,5))
    _, gray = cv2.threshold(gray,gray.max()-1,gray.max(),cv2.THRESH_BINARY)
    #border to decouple image border from obstacle borders
    gray = cv2.copyMakeBorder(gray, 1, 1, 1, 1, cv2.BORDER_CONSTANT, value = 255)
    #resize image
    gray = cv2.resize(gray, desired_size, interpolation = cv2.INTER_AREA)
    cv2.imwrite(savePath, gray)
    if plot:
        img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        img = cv2.resize(img, desired_size, interpolation = cv2.INTER_AREA)
        plt.imshow(img,cmap = 'gray')
        plt.title("Original Image")
        
    return gray

def unzipContour(contour):
    ''' 
    unzip the contour for easier handling and plotting
    
    Variables : contour: 
                
    Return : coordx: array (1xM), M corners coordinates in dim x
             coordy: array (1xM), M corners coordinates in dim y
    
    '''
    coordx = []
    coordy = []
    #for each point of the obstacle, unzip for easier handling
    for i in range(len(contour)):
        x , y= zip(*contour[i])
        x, = x
        y, = y
        coordx.append(x)
        coordy.append(y) 
        
    return coordx, coordy

def resizeImg(img, desired_size):
    '''
    Normalise the map size given the size of the physical map and the max values of the vision map
    
    variables:  coordx: array of all points in graph in x dimension
                coordy: array of all points in graph in y dimension
                
    return: corrx: array of corrected coordinates in x
            corry: array of corrected coordinates in y
    '''
    cv2.resize(img, desired_size)
    
    return corrx , corry

### Find contours and vertices from image
Find the contours and vertices for each obstacle in the image

In [60]:
def findCorners(gray, smoothing_factor, plot = False):
    '''
    Finds corners of obstacles in image
    
    Variables : gray: grayscale image
                smoothing_factor: float, how much the contour will be smoothed
                plot: bool, plot or not the image (default: False)
    

    Return : ###corners: N arrays of 2d (x,y) coords of M corners (Nx2xM)
    '''
    #find the contours of each shape
    contours,_ = cv2.findContours(gray, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    #remove contour of image
    contours.pop(0)
    
    if plot:
        plt.title("Found contours of obstacles")
        
    # for each obstacle     
    K = len(contours)
    # init as empty 
    corners = []
    centers = []
    for k in range(K):
        approx = smoothContours(contours[k], smoothing_factor)
        corners.append(approx)
        # c = FindCentroid(contours[k])
        #centers.append(c)
        if plot:
            x,y = unzipContour(approx)
            # add the first element to close the polygon
            x.append(x[0])
            y.append(y[0])
            plt.plot(x, y, color='green', marker='o', linewidth=2, markersize=8)
        
    return corners
    
def smoothContours(contour, smoothing_factor):
    '''
    Smoothes the contours of an obstacle
    
    Variables : contour: ndarray, contains the vertices of the contour
                smoothing_factor: float, how much the contour will be smoothed

    Return : corners:  array of 2d arrays (x,y) giving the M selected corners of an obstacle (2xM)
    '''
    #to calculate smoothing
    epsilon = smoothing_factor*cv2.arcLength(contour,True) #True means we assume the contour to be closed
    #approximated shape
    approx = cv2.approxPolyDP(contour,epsilon,True)

    return approx
    

https://stackoverflow.com/questions/7263621/how-to-find-corners-on-a-image-using-opencv/7263794#7263794

### Find Centroid of an obstacles

In [30]:
def FindGoal(goalPath, savePath, desired_size):
    '''
    Find the goal from csv file
    
    Variables : csvPath : string, path to csv file from the current directory
                savePath : string, path and name of place to save image
                desired_size: array defining the desired size to resize the img
                plot: bool, plot or not the image (default: False)
                

    Return : center: array, centroid point of goal polygon (2x1)
    '''
    gray = csv2gray(goalPath, savePath, desired_size, plot = False)
    center = FindCentroids(gray)

    return center
    
def FindCentroids(gray):
    '''
    Find the centroid of a closed polygon
    
    Variables : contour: array, 2 dimensional corners in array (2xM)

    Return : centroid: array, centroid point of polygon (2x1)
    '''
    contours,_ = cv2.findContours(gray, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    #remove contour of image
    contours.pop(0)
    # choose the lagest area polygon to avoid measuring noise
    areas = [cv2.contourArea(c) for c in contours]
    max_index = np.argmax(areas)
    cnt = contours[max_index]

    M = cv2.moments(cnt)
    cx = int(M['m10']/M['m00'])
    cy = int(M['m01']/M['m00'])

    return [cx,cy]

### Grow obstacles
To avoid the robot crashing into things, we grow the obstacles in accordance to the distance to the wheels

If we consider the robot will always stay outside the convex hull surrounding the obstacle, by choosing the center and growing from there should allow to avoid the obstacles

In [62]:
def GrowObstacles(contours, radius, desired_size, epsilon, plot = False):
    '''
    Grows all obstacles by a margin
    
    Variables : contour: array, 2 dimensional corners in array 2xM 
                radius: float, margin by which we grow the obstacle
                epsilon : float, margin on how much to grow the corners from the max size of map
                desired_size: array defining the desired size to resize the img
                plot: bool, plot or not the image (default: False) 

    Return : corners:  array of 2d arrays (x,y) giving the M grown corners of an obstacle (2xM)
    '''
    if plot:
        plt.title("Grown contours of obstacles")
        #plt.imshow(gray, aspect = 'equal',origin = 'lower', cmap = 'gray')
    
    K = len(contours)
    corners = []
    for k in range(K):
        temp_corners = GrowObstacle(contours[k], radius, epsilon, desired_size, plot)
        corners.append(temp_corners)
    
    return corners
            
def GrowObstacle(contour, radius, epsilon, desired_size, plot = False):
    '''
    Grows an obstacle by a margin
    
    Variables : contour: array, 2 dimensional corners in array 2xM 
                radius: float, margin by which we grow the obstacle
                epsilon : float, margin on how much to grow the corners from the max size of map
                desired_size: array defining the desired size to resize the img
                plot: bool, plot or not the image (default: False) 

    Return : corners:  array of 2d arrays (x,y) giving the M grown corners of an obstacle (2xM)
    '''
    x = []; y = []
    #unzip coords for easier handling
    x, y  = unzipContour(contour)
    # init empty matrices
    coordx = []
    coordy = []
    
    for i in range(len(x)-1):
        d1 = FindDirection([x[i],y[i]], [x[i-1],y[i-1]])
        d2 = FindDirection([x[i],y[i]], [x[i+1],y[i+1]])
        dx = radius*(d1[0]+d2[0])/math.sqrt(2)
        dy = radius*(d1[1]+d2[1])/math.sqrt(2)
        
        pt = PlacePoints([x[i],y[i]], [dx, dy], contour)
        pt = PointOutOfMap(pt, desired_size, epsilon)
        coordx.append(pt[0])
        coordy.append(pt[1])
    
    # for last corner
    d1 = FindDirection([x[-1],y[-1]], [x[-2],y[-2]])
    d2 = FindDirection([x[-1],y[-1]], [x[0],y[0]])
    dx = radius*(d1[0]+d2[0])/math.sqrt(2)
    dy = radius*(d1[1]+d2[1])/math.sqrt(2)

    pt = PlacePoints([x[-1],y[-1]], [dx, dy], contour)
    pt = PointOutOfMap(pt, desired_size, epsilon)
    coordx.append(pt[0])
    coordy.append(pt[1])
    
    if plot:
        x = coordx
        y = coordy
        x.append(coordx[0])
        y.append(coordy[0])
        plt.plot(x,y , color='red', marker='o', linewidth=3, markersize=1)
    return [coordx, coordy]
    
def FindDirection(pt1,pt2):
    '''
    Normalised direction vector between two points
    
    Variables : pt1 : array, point 1 (2x1)
                pt2 : array, point 2 (2x1)

    Return : direction:  array, normalised direction vector (2x1)
    '''
    
    norm2 = math.sqrt((pt1[0]-pt2[0])**2+(pt1[1]-pt2[1])**2)
    vectx = (1/norm2)*(pt1[0]-pt2[0])
    vecty = (1/norm2)*(pt1[1]-pt2[1])
    return [vectx, vecty]

def PlacePoints(pt, d, contours):
    '''
    Find where to put point to correct for non-linearity
    
    Variables : pt : array, point to test (2x1)
                d : array, distance to check (2x1)
                contours
                

    Return : out: array, new point position (2x1) 
    '''
    test = [pt[0]+d[0], pt[1]+d[1]]
    if (cv2.pointPolygonTest(contours, test, False)) != 1:
        return test
    test = [pt[0]-d[0], pt[1]-d[1]]
    if cv2.pointPolygonTest(contours, test, False) != 1:
        return test
    test = [pt[0]-d[0], pt[1]+d[1]]
    if cv2.pointPolygonTest(contours, test, False) != 1:
        return test
    test = [pt[0]+d[0], pt[1]-d[1]]
    if cv2.pointPolygonTest(contours, test, False) != 1:
        return test
    
    return

def PointOutOfMap(pt, des_size, epsilon):
    '''
    Grows the corners at the edge of the map to make it not advantageous to exit map
    
    Variables : pt: ndarray, contains the vertices of the contour
                des_size : array defining the size of the map
                epsilon : float, margin on how much to grow the corners from the max size of map

    Return : pt: grown point
    '''
    x = pt[0]
    y = pt[1]
    limy = des_size[1]
    if y < 0 :
        y = y - epsilon 
    elif y > limy :
        y = y + epsilon 
    return [x,y]

### Shortest path computation
Using the pyvisgraph library, we can compute the shortest path by using the Dijkstra algorithm on the distances between the nodes. All this is done by the library

link https://github.com/TaipanRex/pyvisgraph


In [64]:
import pyvisgraph as vg

def AddNodesToPolygon(poly, corners):
    '''
    Add nodes to polygon in sense of the pyvisgraph library
    
    Variables : poly: polygon in the pyvisgraph library
                corners: list of coordinates of the corners
                

    Return : poly: filled polygon in the pyvisgraph library
    '''
    x = corners[0]
    y = corners[1]
    for i in range(len(x)-1):
        poly.append(vg.Point(x[i], y[i]))
    return poly

def BuildGraph(corners, graph_name):
    '''
    Build a graph called graph_name containing all the visibility graph
    of the pyvisgraph librairy
    
    Variables : corners: array (N polygons) of lists(M vertices) of coordinates of the corners (Nx2xM)
                graph_name: string giving the name to the graph
            
    Return :
    '''
    #for each polygon
    polys = []
    for k in range(len(corners)):
        poly = []
        poly = AddNodesToPolygon(poly, corners[k])
        polys.append(poly)
        
    g = vg.VisGraph()
    g.build(polys)
    g.save(graph_name)    
    return


## Online Functions
### Find Shortest path from visibility graph

In [36]:
#import pyvisgraph as vg
def FindShortestPath(graph_name, start, stop):
    '''
    Finds shortest path of visibility graph graph_name from start to stop
    
    Variables : graph_name: string giving the name to the graph to load
                start : array [2x1] giving the start position
                stop : array [2x1] giving the goal
                
    Return : path: array of nodes to pass through to avoid hitting obstacles and follow the shortest path
    '''
    g = vg.VisGraph()
    g.load(graph_name)
    #test if in polygon
    if g.point_in_polygon(vg.Point(stop[0],stop[1])) == 1:
        print("Goal is in a polygon")
    if g.point_in_polygon(vg.Point(start[0],start[1])) == -1:
        shortest = g.shortest_path(vg.Point(start[0],start[1]), vg.Point(stop[0], stop[1]))
        path = [[point.x, point.y] for point in shortest]
        state = 0
        return path, state
    else:
        path = []
        state = 1
        return path, state